In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter
import tensorflow as tf

import os
import pickle
import re
from tensorflow.python.ops import math_ops


# 原始数据文档
* https://tianchi.aliyun.com/datalab/dataSet.html?dataId=408


### 样本骨架 数据结构
<img src="assets/sample_skeleton.jpg"/>

### Common Feature 数据结构
<img src="assets/common_feature.jpg"/>

## Common Feature表处理（随机抽取2.5%的UserID和对应的MD5)
MD5的计算逻辑是MD5(userID+commonfeature+page)，同一个用户一天浏览多个page，所以，有多个MD5
### Training Set 所有的UID汇总

In [ ]:
#-*- coding:utf-8 -*-
f = open("xxxxx/sample_train/common_features_train.csv",'r')   
train_total_uid_set = set()
train_total_md5_set = set()
index = 0
for line in f:
    tokens = line.strip().split(",")
    value=0
    md5 = tokens[0]
    feature_arr = tokens[2].split('\001')
    for fea_kv in feature_arr:
        fea_field_id = fea_kv.split('\002')[0]
        fea_id_val = fea_kv.split('\002')[1]
        fea_id = fea_id_val.split('\003')[0]
        fea_val = fea_id_val.split('\003')[1]
        if fea_field_id == '101':
            train_total_uid_set.add(fea_id);
            train_total_md5_set.add(md5)
            break;
    index += 1
    if index % 10000 == 0:
        print("current_index:",index)
f.close()
f_o.close()

pickle.dump(train_total_uid_set, open('./ctr_cvr_data/train_total_uid_set.p', 'wb'))
pickle.dump(train_total_md5_set, open('./ctr_cvr_data/train_total_md5_set.p', 'wb'))
print(0)

In [ ]:
train_total_uid_set = pickle.load(open('./ctr_cvr_data/train_total_uid_set.p', mode='rb'))
train_total_md5_set = pickle.load(open('./ctr_cvr_data/train_total_md5_set.p', mode='rb'))
print(len(train_total_uid_set))
print(len(train_total_md5_set))
count = 0
for k in train_total_uid_set:
    value=0
    for a in k:
        value += ord(a)
    if value%50== 0:
        count = count+1
print(count)

### 训练集Common Feature 2.5%采样

In [ ]:
#-*- coding:utf-8 -*-
f = open("xxxxx/sample_train/common_features_train.csv",'r')   
f_o = open("./ctr_cvr_data/common_features_skeleton_train_sample_2_percent.csv",'w') 
train_uid_set = set()
train_md5_set = set()
index = 0
for line in f:
    tokens = line.strip().split(",")
    value=0
    md5 = tokens[0]
    feature_arr = tokens[2].split('\001')
    for fea_kv in feature_arr:
        fea_field_id = fea_kv.split('\002')[0]
        fea_id_val = fea_kv.split('\002')[1]
        fea_id = fea_id_val.split('\003')[0]
        fea_val = fea_id_val.split('\003')[1]
        if fea_field_id == '101':
            value=0
            # 字符相加
            for a in fea_id:
                value += ord(a)
            # uid mod 40 等价于采样2.5%
            if value%40 == 0:
                f_o.write(line)
                train_uid_set.add(fea_id);
                train_md5_set.add(md5)
            break;
    index += 1
    if index % 10000 == 0:
        print("current_index:",index)
f.close()
f_o.close()

pickle.dump(train_uid_set, open('./ctr_cvr_data/train_uid_set.p', 'wb'))
pickle.dump(train_md5_set, open('./ctr_cvr_data/train_md5_set.p', 'wb'))
print(0)

In [ ]:
train_uid_set = pickle.load(open('./ctr_cvr_data/train_uid_set.p', mode='rb'))
train_md5_set = pickle.load(open('./ctr_cvr_data/train_md5_set.p', mode='rb'))
print(len(train_uid_set))
print(len(train_md5_set))

### Test Set Common Feature 2.5% 采样

In [ ]:
#-*- coding:utf-8 -*-
f = open("xxxxx/sample_test/common_features_test.csv",'r')   
f_o = open("./ctr_cvr_data/common_features_skeleton_test_sample_2_percent.csv",'w') 

test_uid_set = set()
test_md5_set = set()
index = 0
for line in f:
    tokens = line.strip().split(",")
    value=0
    md5 = tokens[0]
    feature_arr = tokens[2].split('\001')
    for fea_kv in feature_arr:
        fea_field_id = fea_kv.split('\002')[0]
        fea_id_val = fea_kv.split('\002')[1]
        fea_id = fea_id_val.split('\003')[0]
        fea_val = fea_id_val.split('\003')[1]
        if fea_field_id == '101':
            value=0
             # 字符相加
            for a in fea_id:
                value += ord(a)
            # uid mod 40 等价于采样2.5%
            if value%40 == 0:
                f_o.write(line)
                test_uid_set.add(fea_id);
                test_md5_set.add(md5)
            break;
    index += 1
    if index % 10000 == 0:
        print("current_index:",index)
f.close()
f_o.close()

pickle.dump(test_uid_set, open('./ctr_cvr_data/test_uid_set.p', 'wb'))
pickle.dump(test_md5_set, open('./ctr_cvr_data/test_md5_set.p', 'wb'))
print(0)

In [ ]:
test_uid_set = pickle.load(open('./ctr_cvr_data/test_uid_set.p', mode='rb'))
test_md5_set = pickle.load(open('./ctr_cvr_data/test_md5_set.p', mode='rb'))
print(len(test_uid_set))
print(len(test_md5_set))


In [ ]:
# Train&Test uid 交集
uid_common = train_uid_set & test_uid_set
md5_common = train_md5_set & test_md5_set
print(len(uid_common))
print(len(md5_common))

## 样本骨架处理（抽取上面有效md5对应的样本)
### 训练骨架

In [ ]:
#-*- coding:utf-8 -*-
f = open("xxxxx/ESMM开源/sample_train/sample_skeleton_train.csv",'r')   
f_o = open("./ctr_cvr_data/sample_skeleton_train_sample_2_percent.csv",'w') 

index = 0
for line in f:
    tokens = line.strip().split(",")
    if tokens[3] in train_md5_set:
        f_o.write(line)
    index += 1
    if index % 1000000 == 0:
        print("current_index:",index)
f.close()
f_o.close()
print(0)

### 测试骨架

In [ ]:
#-*- coding:utf-8 -*-
f = open("xxxx/ESMM开源/sample_test/sample_skeleton_test.csv",'r')   
f_o = open("./ctr_cvr_data/sample_skeleton_test_sample_2_percent.csv",'w') 
index = 0
for line in f:
    tokens = line.strip().split(",")
    if tokens[3] in test_md5_set:
        f_o.write(line)
    index += 1
    if index % 1000000 == 0:
        print("current_index:",index)
f.close()
f_o.close()
print(0)